In [1]:
import cv2
import os
import numpy as np
import scipy.stats as ss
import torch
import time
import matplotlib.pyplot as plt
skin_mean = np.array([113.82950865, 146.5547067 , 203.94020505])
skin_var = np.array([1739.0092018, 1290.22894013, 1426.94384388])
nonSkin_mean = np.array([127.98836827, 128.85303495, 102.16741058])
nonSkin_var = np.array([4390.21815255, 4127.27179659, 4125.52420851])
skin_distbn = ss.multivariate_normal(mean = skin_mean, cov = skin_var)
nonSkin_distbn = ss.multivariate_normal(mean = nonSkin_mean, cov = nonSkin_var)

In [2]:
rootdir = "dataset/train/c0"
imgs = []
file_list = []
for subdir, dirs, files in os.walk(rootdir):
    for f in files:
        imgs.append(cv2.imread(os.path.join(subdir, f)))
        file_list.append(f)

In [3]:
if not os.path.exists('skin_segmented_output/train/c0'):
    os.makedirs('skin_segmented_output/train/c0')

In [4]:
out_dir = 'skin_segmented_output/train/c0'

In [5]:
index = 0
while index+200 <len(imgs):
    print "Hello"
    imgs_batch = imgs[index:index+200]
    time1 = time.time()
    imgs_np = np.array(imgs_batch, dtype = np.double)
    imgs_tensor = torch.from_numpy(imgs_np)
    val1 = skin_distbn.pdf(imgs_tensor)
    val2 = skin_distbn.pdf(imgs_tensor) + nonSkin_distbn.pdf(imgs_tensor)
    val = np.divide(val1,val2)
    time2 = time.time()
    print ("Time taken is ",time2-time1,"seconds")
    
    thresh = 0.5
    bool_mat = val> thresh
    bool_mat = np.array(bool_mat,dtype = np.int8)
    mask = 1- bool_mat
    mask_3d = np.repeat(mask[:, :, :,np.newaxis], 3, axis=3)
    result = mask_3d* imgs_np + [0,255,204] * (1-mask_3d) * imgs_np
    
    for i,item in enumerate(result):
        cv2.imwrite(os.path.join(out_dir,file_list[i+index]),item)
    index+=200
    #print index

Hello
('Time taken is ', 41.28114604949951, 'seconds')


In [6]:
imgs_batch = np.array(imgs[index:])
time1 = time.time()
imgs_np = np.array(imgs_batch, dtype = np.double)
imgs_tensor = torch.from_numpy(imgs_batch)
val1 = skin_distbn.pdf(imgs_tensor)
val2 = skin_distbn.pdf(imgs_tensor) + nonSkin_distbn.pdf(imgs_tensor)
val = np.divide(val1,val2)
time2 = time.time()
print ("Time taken is ",time2-time1,"seconds")

thresh = 0.5
bool_mat = val> thresh
bool_mat = np.array(bool_mat,dtype = np.int8)
mask = 1- bool_mat
mask_3d = np.repeat(mask[:, :, :,np.newaxis], 3, axis=3)
result = mask_3d* imgs_np + [0,255,204] * (1-mask_3d) * imgs_np

for i,item in enumerate(result):
    cv2.imwrite(os.path.join(out_dir,file_list[i+index]),item)


('Time taken is ', 17.01030993461609, 'seconds')
